Change to Tensorflow:V1 Environment 

In [2]:
import tensorflow as tf
import onnx
from onnx_tf.backend import prepare
import quaternion
import pandas
import numpy as np
import matplotlib.pyplot as plt
from metric import compute_ate_rte
from onnx import helper
import time


/home/u22/anaconda3/envs/tf_env/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
def convert_onnx_tflite (onnx_path , tflite_path , graph_path):
    model = onnx.load(onnx_path)

    # modified code

    # Define a mapping from old names to new names
    name_map = {"input.1": "input_1"}

    # Initialize a list to hold the new inputs
    new_inputs = []

    # Iterate over the inputs and change their names if needed
    for inp in model.graph.input:
        if inp.name in name_map:
            # Create a new ValueInfoProto with the new name
            new_inp = helper.make_tensor_value_info(name_map[inp.name],
                                                    inp.type.tensor_type.elem_type,
                                                    [dim.dim_value for dim in inp.type.tensor_type.shape.dim])
            new_inputs.append(new_inp)
        else:
            new_inputs.append(inp)

    # Clear the old inputs and add the new ones
    model.graph.ClearField("input")
    model.graph.input.extend(new_inputs)

    # Go through all nodes in the model and replace the old input name with the new one
    for node in model.graph.node:
        for i, input_name in enumerate(node.input):
            if input_name in name_map:
                node.input[i] = name_map[input_name]

    # Save the renamed ONNX model
    onnx.save(model, 'new_model.onnx')
    
    tf_rep = prepare(model)

    
    tf_rep.export_graph(graph_path)
   
    converter = tf.lite.TFLiteConverter.from_saved_model(graph_path)
    
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]      
    # tell converter which type of optimization techniques to use
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tfmodel = converter.convert() 
    
    # Save the model.
    with open(tflite_path, 'wb') as f:
      f.write(tfmodel)
    
    return tfmodel
